In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/titanic/titanic_train.csv')

In [ ]:
df.head()

# Exploratory Data Analysis 

Check out the missing data

### Missing Data 

In [ ]:
df.isnull()

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False)

Roughly 20 percent of the Age data is missing. The proportion of Age missing is likely small enough for reasonable replacement with some form of imputation. Looking at the Cabin column, it looks like we are just missing too much of that data to do something useful with at a basic level. We'll probably drop this later, or change it to another feature like "Cabin Known: 1 or 0"

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x=df['Survived'])

In [ ]:
sns.countplot(x=df['Survived'],hue=df['Sex'],palette='RdBu_r')

In [ ]:
sns.countplot(x=df['Survived'],hue=df['Pclass'],palette='rainbow')

In [ ]:
sns.distplot(df['Age'].dropna(),kde=False)

In [ ]:
df['Age'].hist(alpha=0.3)

In [ ]:
sns.countplot(x=df['SibSp'],palette='rainbow')

In [ ]:
df['Fare'].hist(bins=40,figsize=(8,4))

In [ ]:

import cufflinks as cf
cf.go_offline()
df['Fare'].iplot(kind='hist',bins=30,color='green')

### Data Cleaning

We want to fill in missing age data instead of just dropping the missing age data rows. One way to do this is by filling in the mean age of all the passengers (imputation). However we can be smarter about this and check the average age by passenger class. For example:

In [ ]:
plt.figure(figsize=(12,7))
sns.boxplot(x=df['Pclass'],y=df['Age'],palette='winter')

We can see the wealthier passengers in the higher classes tend to be older, which makes sense. We'll use these average age values to impute based on Pclass for Age.

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        if Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

In [ ]:
df['Age'] = df[['Age','Pclass']].apply(impute_age,axis=1)

In [ ]:
sns.heatmap(data=df.isnull(), yticklabels=False)

Drop cabin column as most of the data is missing

In [ ]:
df.drop('Cabin',axis=1, inplace=True)

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)

### Converting Categorical Features 

We'll need to convert categorical features to dummy variables using pandas! Otherwise our machine learning algorithm won't be able to directly take in those features as inputs.

In [ ]:
df.info()

In [ ]:
sex = pd.get_dummies(df['Sex'], drop_first=True)
embark = pd.get_dummies(df['Embarked'], drop_first=True)

In [ ]:
df.drop(['Sex','Embarked','PassengerId','Name','Ticket'], axis=1, inplace=True)

In [ ]:
df.head()

## Building a Logistic Regression model

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('Survived', axis=1), df['Survived'], test_size=0.3, random_state = 101)

### Training and Predicting 

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

In [ ]:
predictions = logmodel.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
accuracy = confusion_matrix(y_test, predictions)

In [ ]:
print(accuracy)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print(accuracy)

### Evaluation 

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))